In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
weekend_Jul_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in weekend_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

25920

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
x[:,:2]

matrix([[ 336752.83005355,  338352.11152615],
        [ 351081.52796039,  356072.37715531],
        [ 624819.17424071,  616402.34285139],
        [ 696729.97210139,  678981.37087328],
        [ 173440.06461621,  174520.83949704],
        [ 210989.63528051,  214716.87762918],
        [ 411376.59787012,  415763.388488  ],
        [ 419995.80934705,  408775.11594838],
        [ 393364.54516856,  381905.72280156],
        [ 390106.08744141,  402705.90436935],
        [ 374319.78936257,  380768.75741976],
        [ 316670.36910445,  332595.54254673],
        [  95513.88873282,   94198.86067832],
        [ 105126.91116042,  100163.26835863],
        [ 155827.9775564 ,  148908.76999296],
        [ 143087.69344729,  128552.99340808],
        [ 217670.39879191,  216223.56693787],
        [ 217154.78149655,  216558.33179241],
        [ 365115.3421185 ,  372794.48434694],
        [ 323055.95616159,  333251.57909773],
        [ 529657.80300225,  517077.28243598],
        [ 519840.12097395,  517660

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 1080)

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e-02, 9e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.15255358e+07  0.00000000e+00  5.03e+03 1.17e+03  1.00e+06     0s
   1  -6.16650733e+06 -4.33269316e+06  1.06e+03 2.48e+02  2.42e+05     0s
   2  -4.60469272e+06 -7.50055640e+06  1.06e-03 2.48e-04  5.17e+04     0s
   3  -5.25101435e+06 -5.51143458e+06  1.58e-05 3.69e-06  4.6

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.2235800811650188e-08,
 42705.417343278896,
 31768.62491515207,
 25218.443007523867,
 47551.49184274774,
 163582.05128469673,
 114409.63851685883,
 1.4031538148683214e-07,
 1.372540179583016e-09,
 260194.66224446098,
 1.3725397439148622e-09,
 1.3725399466905032e-09,
 1.372540070522015e-09,
 1.3817883771514293e-09,
 179955.26400238724,
 4.754290050722487e-09,
 3.2024898431528826e-07,
 123248.69936553434,
 181367.95739589556,
 5.842517918925715e-05,
 0.0004079777575781724,
 200831.3905532728,
 154052.8646898536,
 6.685038487090191e-07,
 52018.26291014948,
 31657.50721511805,
 2.8439946080774598e-05,
 141279.47244208146,
 158153.9442423241,
 4.754290302942707e-09,
 6.242410421697779e-07,
 66429.35346867538,
 143728.98034212508,
 1.4393142708692475e-06,
 0.00043791111374058035,
 237605.70015698753,
 4.754291467298952e-09,
 6.233595996210347e-07,
 5.662927546390824e-06,
 114371.47017303112,
 499404.6070039869,
 0.0013114006186109861,
 1.6115362646048666e-05,
 4.752900021067136e-09,
 7.451